# Airbnb Rio de Janeiro - Regressão lognormal

# 1. Introdução

Esse notebook propõe aplicar um modelo de Regressão Lognormal para os [dados do Airbnb na cidade do Rio de Janeiro](https://insideairbnb.com/get-the-data/#:~:text=Rio%20de%20Janeiro%2C%20Rio%20de%20Janeiro%2C%20Brazil). 

Os dados são carregados do processo de [feature engineering](https://github.com/BPRateiro/rio-airbnb-data-analysis/blob/main/notebooks/feature_engineering.ipynb).Incluem informações detalhadas sobre a disponibilidade de listagens, características dos imóveis, interações de usuários, e preços. 

O objetivo principal é construir um modelo que preserva sua interpretabilidade.

## 1.1 Configurações e bibliotecas

In [5]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

## 1.2 Leitura dos dados

Removemos também colunas que tem apenas um valor

In [3]:
df_inicial = pd.read_parquet('../data/silver/feature_engineering_output.parquet')
df_inicial.drop(columns=[col for col in df_inicial.columns if df_inicial[col].nunique() == 1], inplace=True)
df_inicial.head()

,accommodates,availability_365,availability_60,bathrooms,bedrooms,beds,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,days_since_first_review,...,review_scores_value,reviews_per_month,verification_email,verification_work_email,was_reviewed,host_response_time_within_a_day,host_response_time_within_a_few_hours,host_response_time_within_an_hour,room_type_others,room_type_private_room
0,5.0,269.0,59.0,2.0,1.0,2.0,1.0,0.0,0.0,131.8,...,4.502,1.700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,176.0,56.0,2.0,2.0,0.0,6.0,0.0,0.0,173.0,...,4.500,0.690,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,2.0,267.0,59.0,1.0,1.0,1.0,0.0,2.0,0.0,151.4,...,4.600,0.448,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,2.0,38.0,35.0,1.0,1.0,1.0,1.0,1.0,0.0,531.0,...,5.000,0.230,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,5.0,10.0,10.0,2.0,2.0,3.0,1.0,0.0,0.0,3631.0,...,4.740,3.040,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


# 2. Modelo de Baseline

Embora no EDA vimos que a distribuição mais adequada fosse a Gen Extreme, por razões de uma maior facilidade ao modelar e interpretar os resultados do modelo, optamos aqui por modelar nossos dados com uma distribuição Gamma.

O modelo de baseline corresponde ao ajuste da média à todo e qualquer ponto da distribuição. Portanto, o modelo de baseline consiste ao intercepto, sem considerar outras variáveis explicativas.

In [7]:
# Primeiro, adicionamos uma constante para representar o intercepto
X_baseline = sm.add_constant(np.ones(df_inicial.shape[0]))  # Apenas o intercepto
y = df_inicial['price']

# Ajustar o modelo GLM com distribuição Gama e função de ligação log
gamma_model = sm.GLM(y, X_baseline, family=sm.families.Gamma(link=sm.families.links.log()))
gamma_result = gamma_model.fit()

# Comparando o coeficiente encontrado com a média dos preços
print(y.mean(), np.exp(5.8762))

# Resumo do modelo ajustado
gamma_result.summary()

356.4525252525253 356.45214670390556


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  price   No. Observations:                24255
Model:                            GLM   Df Residuals:                    24254
Model Family:                   Gamma   Df Model:                            0
Link Function:                    log   Scale:                         0.43221
Method:                          IRLS   Log-Likelihood:            -1.6193e+05
Date:                Tue, 13 Aug 2024   Deviance:                       9770.7
Time:                        19:40:36   Pearson chi2:                 1.05e+04
No. Iterations:                     7   Pseudo R-squ. (CS):              0.000
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.8762      0.004   1392.028      0.000       5.868       5.884
==============================================================================
"""

# 3. Modelo inicial

Vimos durante o EDA que `price` melhor se adequa à distribuição lognormal, considerando todos os valores de price como reais. Utilizaremos um modelo linear generalizado (GLM) adequado à variável resposta, evitando-se transformações nas variáveis e preservando a interpretabilidade.

In [13]:
# Seleção de algumas variáveis explicativas para o modelo
X = df_inicial.drop(columns=['price'])

# Adicionar uma constante para o intercepto
X = sm.add_constant(X)

# Ajustar o modelo GLM com as variáveis explicativas selecionadas
gamma_model_with_explanatory = sm.GLM(y, X, family=sm.families.Gamma(link=sm.families.links.log()))
gamma_result_with_explanatory = gamma_model_with_explanatory.fit()

# Resumo do modelo ajustado
gamma_result_with_explanatory.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  price   No. Observations:                24255
Model:                            GLM   Df Residuals:                    24211
Model Family:                   Gamma   Df Model:                           43
Link Function:                    log   Scale:                         0.25149
Method:                          IRLS   Log-Likelihood:            -1.5395e+05
Date:                Tue, 13 Aug 2024   Deviance:                       5154.1
Time:                        19:48:19   Pearson chi2:                 6.09e+03
No. Iterations:                    22   Pseudo R-squ. (CS):             0.5308
Covariance Type:            nonrobust                                         
================================================================================================================
                                                   coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
const                                          -65.4302      2.291    -28.561      0.000     -69.920     -60.940
accommodates                                     0.0475      0.002     19.274      0.000       0.043       0.052
availability_365                                 0.0004   3.32e-05     11.206      0.000       0.000       0.000
availability_60                                  0.0009      0.000      4.746      0.000       0.001       0.001
bathrooms                                        0.1057      0.006     18.275      0.000       0.094       0.117
bedrooms                                         0.1641      0.005     31.183      0.000       0.154       0.174
beds                                            -0.0192      0.002     -7.795      0.000      -0.024      -0.014
calculated_host_listings_count_entire_homes      0.0001      0.000      0.569      0.569      -0.000       0.000
calculated_host_listings_count_private_rooms     0.0012      0.002      0.609      0.543      -0.003       0.005
calculated_host_listings_count_shared_rooms     -0.0283      0.004     -6.912      0.000      -0.036      -0.020
days_since_first_review                      -4.814e-05   5.21e-06     -9.239      0.000   -5.84e-05   -3.79e-05
days_since_host_active                        2.101e-05   3.13e-06      6.712      0.000    1.49e-05    2.71e-05
days_since_last_review                           0.0002   1.05e-05     19.442      0.000       0.000       0.000
description_length                           -5.651e-06   2.16e-05     -0.261      0.794   -4.81e-05    3.68e-05
host_about_length                            -5.064e-05   1.15e-05     -4.417      0.000   -7.31e-05   -2.82e-05
host_acceptance_rate                             0.0362      0.015      2.384      0.017       0.006       0.066
host_identity_verified                          -0.0010      0.009     -0.111      0.912      -0.019       0.017
host_is_superhost                                0.0315      0.008      3.822      0.000       0.015       0.048
host_listings_count                              0.0002   4.53e-05      5.412      0.000       0.000       0.000
host_responded                                  -0.3694      0.016    -23.141      0.000      -0.401      -0.338
instant_bookable                                 0.0450      0.009      5.271      0.000       0.028       0.062
latitude                                        -4.7681      0.104    -45.775      0.000      -4.972      -4.564
longitude                                        0.9097      0.038     24.160      0.000       0.836       0.984
maximum_nights                               -3.832e-05   8.16e-06     -4.695      0.000   -5.43e-05   -2.23e-05
minimum_nights                  

# 4. Avaliação das premissas

In [7]:
from statsmodels.stats.stattools import durbin_watson
from scipy.stats import shapiro, kstest
from statsmodels.stats.diagnostic import het_breuschpagan, het_white

# Obter resíduos
residuals = glm_model.resid_response

# Teste de Independência dos Erros
dw_statistic = durbin_watson(residuals)
print(f"Estatística de Durbin-Watson: {dw_statistic}")

# Teste de Normalidade dos Resíduos
ks_stat, ks_p = kstest(residuals, 'norm', args=(residuals.mean(), residuals.std()))
print(f"Teste de Kolmogorov-Smirnov: estatística = {ks_stat}, valor-p = {ks_p}")

# Teste de Homoscedasticidade
bp_test = het_breuschpagan(residuals, glm_model.model.exog)
print(f"Teste de Breusch-Pagan: estatística LM = {bp_test[0]}, valor-p = {bp_test[1]}")

# Adequação do Modelo
print(f"AIC: {glm_model.aic}")

Estatística de Durbin-Watson: 1.9922691778095782
Teste de Kolmogorov-Smirnov: estatística = 0.21312592137636, valor-p = 0.0
Teste de Breusch-Pagan: estatística LM = 2588.453904086076, valor-p = 0.0
AIC: 406111.9547062845
